# Генератор ответов на медицинские вопросы

В этом ноутбуке создадим медицинскую вопросно-ответную систему, обучив GPT на датасете https://huggingface.co/datasets/blinoff/medical_qa_ru_data.


In [1]:
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 17.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [2]:
import numpy as np
import pandas as pd
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import torch
from sklearn.model_selection import train_test_split

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
path =  "/content/drive/My Drive/NLP/Course_Project/"

In [5]:
data = pd.read_csv(path + 'bigdata_medical.csv')
data.dropna(inplace=True)
data

,desc,ans,class
0,"Здравствуйте, беспокоят боли в области сердца,...",Обратиться к врачу нужно.;\nПользователь побла...,1
1,Здравствуйте ! Можно ли заниматься фитнесом ( ...,Можно.;\nПользователь поблагодарил за ответ на...,1
2,"Здравствуйте , когда должна начать болеть груд...","Здравствуйте,она начнёт болеть не сразу и дале...",1
3,Можно ли пить горячий молоко при пневмонии? Да...,Пища и напитки должны быть комфортной температ...,1
4,"Здравствуйте, Я принял таблетку немозола за 15...",Нет.;\nНе повлияет.,1
...,...,...,...
99995,"Здравствуйте, скажите, делают ли операции на с...","Если она необходима,то делается;\nЭто решается...",1
99996,Добрый день.Подскажите пожалуйста что может бы...,"Здравствуйте, вероятнее всего,что у вас болит ...",1
99997,"Здравствуйте! Ребёнку три года,появилась залож...",Можно для облегчения отхождения мокроты.;\nЗдр...,1
99998,Здравствуйте 10 дней назад делали операцию на ...,Все вопросы об отмене препарата и изменении до...,1


Разобьем датасет на вопросы и ответы, тренировочную и тестовую выборки:

In [6]:
train_q, test_q, train_ans, test_ans = train_test_split(data.desc,
                                                        data.ans,
                                                        test_size=.2,
                                                        random_state=42)

In [7]:
device = torch.device("cuda")

Скачаем модель GPT от сбер:

In [8]:
model_name = 'sberbank-ai/rugpt3small_based_on_gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/551M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [12]:
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

Функции для простого препроцессинга (удаление непечатных знаков) и сборки датасета для обучения. При сборке датасета вопрос помечаем как '\nx:', а ответ - как '\ny:'

In [9]:
def preproc_text(text):
    res = str(text).strip()
    res = re.sub(r"\s+", " ", res)
    return res

def build_data(data_q, data_ans):
    data = []
    for idx, texts in enumerate(data_q):
        question = preproc_text(texts)
        answer = preproc_text(data_ans.iloc[idx])
        res = '\nx:' + question + '\ny:' + answer
        data.append(res)
    return data

In [10]:
train_data = build_data(train_q, train_ans)
test_data = build_data(test_q, test_ans)

In [11]:
tokenizer.pad_token = tokenizer.eos_token

Токенизируем тренировочный и тестовый датасеты:

In [ ]:
train_encodings = tokenizer(train_data, padding='max_length', truncation=True, max_length=512, return_tensors='pt')
test_encodings = tokenizer(test_data, padding='max_length', truncation=True, max_length=512, return_tensors='pt')

Класс для создания torch-датасета:

In [ ]:
class QAMedicalDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = QAMedicalDataset(train_encodings)
test_dataset = QAMedicalDataset(test_encodings)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Зададим аргуметы для дообучения модели:

In [ ]:
training_args = TrainingArguments(
    output_dir="./data_medical", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=1, # number of training epochs
    per_device_train_batch_size=4, # batch size for training
    per_device_eval_batch_size=4,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

Обучим модель:

In [ ]:
trainer.train()

<ipython-input-13-b29274360c54>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss
500,4.447800
1000,3.665800
1500,3.677300
2000,3.713100


Checkpoint destination directory ./data_medical/checkpoint-800 already exists and is non-empty.Saving will proceed but saved results may be invalid.
<ipython-input-13-b29274360c54>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Checkpoint destination directory ./data_medical/checkpoint-1600 already exists and is non-empty.Saving will proceed but saved results may be invalid.
<ipython-input-13-b29274360c54>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


TrainOutput(global_step=2000, training_loss=3.876008728027344, metrics={'train_runtime': 1270.7256, 'train_samples_per_second': 6.296, 'train_steps_per_second': 1.574, 'total_flos': 2090336256000000.0, 'train_loss': 3.876008728027344, 'epoch': 1.0})

Сохраним модель на диск:

In [ ]:
tokenizer.save_pretrained(path + 'tokenizer_qa_medical_gen')
model.save_pretrained(path + 'model_qa_medical_gen')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(path + 'tokenizer_qa_medical_gen')
model = AutoModelForCausalLM.from_pretrained(path + 'model_qa_medical_gen')

Функция для ведения диалога. Модель "держит в голове" контекст последних 10-ти сообщений.

In [ ]:
def respond_to_dialog(texts):
    prefix = '\nx:'
    for i, t in enumerate(texts):
        prefix += t
        prefix += '\nx:' if i % 2 == 1 else '\ny:'
    tokens = tokenizer(prefix, return_tensors='pt')
    tokens = {k: v.to(model.device) for k, v in tokens.items()}
    end_token_id = tokenizer.encode('\n')[0]
    size = tokens['input_ids'].shape[1]
    output = model.generate(
        **tokens,
        eos_token_id=end_token_id,
        do_sample=True,
        max_length=size+128,
        repetition_penalty=3.2,
        temperature=1,
        num_beams=3,
        length_penalty=0.01,
        pad_token_id=tokenizer.eos_token_id
    )
    decoded = tokenizer.decode(output[0])
    result = re.findall(r'\ny:(.+)', decoded)[-1]
    return result.strip()

Протестируем генератор:

In [ ]:
seed = input('Пообщайтесь с MedConsultBot\n')
history = [seed]
while True:
    result = respond_to_dialog(history[-10:])
    next_sentence = input(result + '\n')
    if next_sentence == 'Стоп':
        break
    history.append(result)
    history.append(next_sentence)

Пообщайтесь с MedConsultBot
Что делать, если болит горло?
Здравствуйте. Обратитесь к стоматологу.; Сдайте анализ крови на гормоны щитовидной железы; Здравствуйте! При остеохондрозеозргических заболеваниях горла и ЛОР органов грудной клетки нестероидного отделачная гипертирецепным методом в пределахгордиатонефрофизарную терапию при хроническом тонзиллитаризме (обратитесь к терапевту). Выполните МРТ шейного отдела позвоночника с контрастированием стенок сосудов головного мозга, шеи, брюшной полости, пояснично-крестцового отдела позвоночника, грудного отдела позвоночника, межреберного отдела позвоночника
Стоп
